In [10]:
print("Hello World")

Hello World


## Original Image representation in 5*5 array of pixels

In [ ]:
import numpy as np
from PIL import Image

# Given matrix of pixels
pixels = np.array([
    [154, 100, 75, 80, 192],
    [201, 178, 219, 95, 112],
    [48, 86, 173, 60, 104],
    [133, 12, 41, 50, 239],
    [68, 23, 17, 200, 189]
], dtype=np.uint8)

# Create image from pixel array
image = Image.fromarray(pixels)

# Display the image
image.show()

In [ ]:
import numpy as np
from PIL import Image

# Given matrix of grayscale pixels
pixels = np.array([
    [154, 100, 75, 80, 192],
    [201, 178, 219, 95, 112],
    [48, 86, 173, 60, 104],
    [133, 12, 41, 50, 239],
    [68, 23, 17, 200, 189]
], dtype=np.uint8)

# Create image from grayscale pixel array
image = Image.fromarray(pixels, mode='L')

# Save the image as PNG (change the path as per your need)
save_path = r'F:/Research/Sahu Sir/output.png'
image.save(save_path)

# Display the saved message
print(f"The image has been saved at {save_path}")

## Here's a step-by-step process for implementing a robust steganographic algorithm using a 5x5 matrix of pixels:



### Step 1: Encryption
First, we need to encrypt the secret message using a secure encryption algorithm. Let's assume we are using AES encryption. We will use the following key to encrypt the message:

In [16]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
import hashlib

# Generate a 128-bit key using a passphrase
passphrase = "YourSecretPassphrase"
key = hashlib.sha256(passphrase.encode()).digest()[:16]

# Encrypt the secret message
def encrypt_message(message, key):
    cipher = AES.new(key, AES.MODE_ECB)
    padded_message = pad(message, AES.block_size)
    ciphertext = cipher.encrypt(padded_message)
    return ciphertext

# Replace 'Secret message to be encrypted' with the actual secret message
secret_message = b'Secret message to be encrypted'
encrypted_message = encrypt_message(secret_message, key)

### Step 2: Pixel Modification
Now, we will load the image into a 5x5 matrix of pixels.

Next, we will embed the encrypted message bits into the 5x5 matrix of pixels using a pixel modification technique. Let's use LSB substitution.

In [17]:
from PIL import Image
import numpy as np

# Load the steganographic image
image_path = r'F:/Research/Sahu Sir/output.png'  # Path to the grayscale PNG image
steganographic_image = Image.open(image_path)

# Convert the steganographic image to a numpy array
pixel_matrix = np.array(steganographic_image)

# Convert the pixel matrix and encrypted message into binary representations
def to_binary(image):
    binary_image = []
    for row in image:
        binary_row = []
        for pixel in row:
            binary_pixel = format(pixel, '08b')  # Convert pixel to 8-bit binary
            binary_row.append(binary_pixel)
        binary_image.append(binary_row)
    return binary_image

binary_image = to_binary(pixel_matrix)
binary_message = '011000010110111001100011011101000110010101110010'  # Example encrypted message

# Embed the encrypted message into the pixel matrix using LSB substitution
def embed_message(image, message):
    modified_image = np.copy(image)
    flat_image = modified_image.flatten()
    bit_index = 0
    for i in range(len(flat_image)):
        if bit_index >= len(message):
            break
        pixel = [int(x) for x in format(flat_image[i], '08b')]  # Convert pixel to a list of integers
        pixel[-1] = int(message[bit_index])
        flat_image[i] = int(''.join(map(str, pixel)), 2)  # Convert the modified pixel back to an integer
        bit_index += 1
    modified_image = flat_image.reshape(image.shape)
    return modified_image

steganographic_image = embed_message(pixel_matrix, binary_message)

### Step 3: Decryption and Extraction
Finally, we extract the hidden message from the steganographic image and decrypt it using the secret key.

In [18]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
import hashlib

# Extract the hidden message from the steganographic image
def extract_message(image):
    extracted_message = ""
    for row in image:
        for pixel in row:
            binary_pixel = format(pixel, '08b')  # Convert pixel to 8-bit binary
            extracted_message += binary_pixel[-1]
    return extracted_message

extracted_message = extract_message(steganographic_image)

# Convert the extracted message to bytes
extracted_bytes = bytearray()
for i in range(0, len(extracted_message), 8):
    byte = extracted_message[i:i+8]
    extracted_bytes.append(int(byte, 2))

# Decrypt the extracted message using the secret key
passphrase = "YourSecretPassphrase"  # Example passphrase
key = hashlib.sha256(passphrase.encode()).digest()[:16]

def decrypt_message(ciphertext, key):
    cipher = AES.new(key, AES.MODE_ECB)
    decrypted_data = cipher.decrypt(ciphertext)
    original_message = unpad(decrypted_data, AES.block_size, style='pkcs7')
    return original_message.decode()

# Decrypt the extracted message
decrypted_message = decrypt_message(extracted_bytes, key)

ValueError: Data must be aligned to block boundary in ECB mode

Remember, this is a simplified example using a 5x5 matrix of pixels and AES encryption. In practice, you would need to adapt the code and consider additional security measures based on your specific requirements and the steganalysis techniques you want to protect against.
this is a simplified example using a 5x5 matrix of pixels and AES encryption. 

To summarize, here are the three steps:

Step 1:

Generate a 128-bit key using a passphrase.
Encrypt the secret message using AES in ECB mode with PKCS7 padding.
Step 2:

Load the image and convert it to grayscale.
Convert the pixel matrix and encrypted message to binary representations.
Embed the encrypted message into the matrix of pixels using LSB substitution.
Step 3:

Extract the hidden message from the steganographic image.
Decrypt the extracted message using the secret key and PKCS7 padding.

### Here is the code that incorporates all these steps:

In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib
import numpy as np
from PIL import Image

# Step 1: Generate a 128-bit key using a passphrase
passphrase = "YourSecretPassphrase"
key = hashlib.sha256(passphrase.encode()).digest()[:16]

# Step 2: Embed the encrypted message into the image
def embed_message(image, message):
    modified_image = [row[:] for row in image]
    bit_index = 0
    # Embed the length of the message as the first 16 bits
    message_length = len(message)
    for i in range(2):  # Embed 2 bytes (16 bits)
        byte = (message_length >> (8 * (1 - i))) & 0xFF
        for j in range(8):
            pixel = modified_image[i][j]
            pixel = list(pixel)
            pixel[-1] = str((int(pixel[-1]) & 0xFE) | ((byte >> (7 - j)) & 0x01))
            modified_image[i][j] = ''.join(pixel)
    # Embed the rest of the message
    for row in range(2, 5):
        for col in range(5):
            pixel = modified_image[row][col]
            pixel = list(pixel)
            if bit_index < len(message):
                pixel[-1] = message[bit_index]
                bit_index += 1
            modified_image[row][col] = ''.join(pixel)
    return modified_image

# Step 3: Extract the hidden message from the steganographic image
def extract_message(image):
    extracted_message = ""
    # Extract the length of the message from the first 16 bits
    message_length = 0
    for i in range(2):
        byte = 0
        for j in range(8):
            pixel = image[i][j]
            byte = (byte << 1) | (int(pixel[-1]) & 0x01)
        message_length = (message_length << 8) | byte
    # Extract the rest of the message
    for row in range(2, 5):
        for col in range(5):
            pixel = image[row][col]
            extracted_message += pixel[-1]
    # Truncate the extracted message based on the length
    extracted_message = extracted_message[:message_length]
    return extracted_message

# Step 4: Decrypt the extracted message using the secret key
def decrypt_message(ciphertext, key):
    cipher = AES.new(key, AES.MODE_ECB)
    decrypted_data = cipher.decrypt(ciphertext)
    original_message = unpad(decrypted_data, AES.block_size, style='pkcs7')
    return original_message

# Step 5: Load the image
image_path = 'F:/Research/Sahu Sir/output.png'  # Replace with the actual path to your image
original_image = Image.open(image_path)

# Step 6: Convert the image to grayscale
original_image = original_image.convert('L')

# Step 7: Get the pixel matrix from the image
pixel_matrix = []
for row in range(original_image.size[1]):
    pixel_row = []
    for col in range(original_image.size[0]):
        pixel_row.append(original_image.getpixel((col, row)))
    pixel_matrix.append(pixel_row)


# Step 8: Convert the pixel matrix and encrypted message into binary representations
def to_binary(image):
    binary_image = []
    for row in image:
        binary_row = []
        for pixel in row:
            binary_pixel = format(pixel, '08b')  # Convert pixel to 8-bit binary
            binary_row.append(binary_pixel)
        binary_image.append(binary_row)
    return binary_image

binary_image = to_binary(pixel_matrix)
binary_message = ''.join(format(byte, '08b') for byte in encrypted_message)

# Step 9: Extract the encrypted message from the steganographic image
extracted_message = extract_message(binary_image)

# Step 10: Convert the extracted message from binary to bytes
extracted_bytes = bytes([int(extracted_message[i:i+8], 2) for i in range(0, len(extracted_message), 8)])

# Step 11: Decrypt the extracted message
decrypted_message = decrypt_message(extracted_bytes, key)

# Step 12: Print the decrypted message
print("Decrypted Message:", decrypted_message.decode())

## The steps of the algorithm using a 5x5 matrix of pixels and explain the transformations that occur at each stage.

`Step 1`: Encryption
Assume we have the following 5x5 matrix of pixels as the original image:

```
Original Image:
[ 154, 100,  75,  80, 192 ]
[ 201, 178, 219,  95, 112 ]
[  48,  86, 173,  60, 104 ]
[ 133,  12,  41,  50, 239 ]
[  68,  23,  17, 200, 189 ]
```

Let's say the secret message to be encrypted is "Hello." After encrypting the message using AES encryption, we obtain the encrypted message, which we will use for embedding.

`Step 2`: Pixel Modification
Next, we will embed the encrypted message bits into the least significant bits (LSBs) of the pixel values using LSB substitution.

To illustrate this, let's take the first pixel (154) and its binary representation (10011010). We will substitute the LSBs of this pixel with the first few bits of the encrypted message.

Suppose the first three bits of the encrypted message are "010." The modified pixel would be:

```
Original Pixel: 10011010
Modified Pixel: 1001101[0]
```

Repeat this process for each pixel in the 5x5 matrix, modifying the LSBs based on the bits of the encrypted message until all the message bits are embedded.

`Step 3`: Decryption and Extraction
To retrieve the hidden message from the steganographic image, we reverse the process.

Let's consider the first modified pixel from the previous step (1001101[0]). We extract the LSB and combine it with the other modified pixels to obtain the extracted message.

```
Modified Pixel: 1001101[0]
Extracted Bit: 0
```

By repeating this process for each modified pixel in the steganographic image, we retrieve all the embedded bits of the encrypted message.

Finally, we decrypt the extracted message using the secret key and obtain the original secret message.

This is a simplified explanation of the algorithm and the transformations that occur within a 5x5 matrix of pixels. In practice, the process may involve additional steps, such as data encoding, error correction, and statistical analysis to ensure robustness and security.

# New method

As an AI language model, I can provide you with an outline of an algorithm for steganography that aims to maximize the number of secret bits while maintaining the visual similarity to the original picture. However, it's important to note that steganography can be a complex field, and designing an algorithm that is truly "unbreakable" is extremely challenging. The strength of steganography lies in making it computationally expensive and time-consuming for potential attackers to discover the hidden message.

Here's a basic algorithm that you can consider as a starting point:

1. Preprocessing:
   - Convert the original picture to its binary representation.
   - Generate a secret key known only to the intended recipient.

2. Secret Bit Embedding:
   - Divide the picture into small, non-overlapping blocks of pixels.
   - For each block, extract the least significant bits (LSBs) of the pixel values.
   - Replace the LSBs with the secret bits to be embedded, using a specific embedding technique (e.g., LSB substitution, random LSB substitution, or matrix encoding).
   - Reconstruct the modified picture with the embedded secret bits.

3. Security Enhancement:
   - Apply cryptographic techniques to the secret bits before embedding (e.g., encryption or hashing) to further enhance security.
   - Encrypt the secret bits using the secret key to ensure that only the intended recipient can decrypt and retrieve the message.

4. Visual Similarity Preservation:
   - During the secret bit embedding process, carefully analyze and modify the LSBs to minimize visible changes in the image.
   - Implement techniques such as error diffusion, dithering, or adaptive pixel adjustment to maintain visual similarity between the original and modified images.

5. Secret Bit Extraction:
   - To extract the secret bits, use the same algorithm and the secret key.
   - Reconstruct the modified picture using the same block-based approach as in step 2.
   - Extract and decrypt the embedded secret bits using the secret key.

Remember that the security of any steganographic algorithm depends on various factors, including the embedding technique, key management, encryption strength, and the computational resources available to potential attackers. It's always advisable to consult with experts in the field and conduct thorough testing to ensure the algorithm's effectiveness.

In [ ]:
from PIL import Image
import random

def embed_secret_bits(image_path, secret_bits, key):
    # Load the original image
    image = Image.open(image_path)
    pixels = image.load()

    # Convert the secret bits to a list of integers
    secret_integers = [int(bit) for bit in secret_bits]

    # Generate a random seed using the secret key
    random.seed(key)

    # Embed the secret bits
    width, height = image.size
    for y in range(height):
        for x in range(width):
            r, g, b = pixels[x, y]

            # Embed one secret bit in each color channel
            if secret_integers:
                secret_bit = secret_integers.pop(0)
                r = (r & 0xFE) | secret_bit

            if secret_integers:
                secret_bit = secret_integers.pop(0)
                g = (g & 0xFE) | secret_bit

            if secret_integers:
                secret_bit = secret_integers.pop(0)
                b = (b & 0xFE) | secret_bit

            pixels[x, y] = (r, g, b)

    # Save the modified image
    modified_image_path = 'modified_' + image_path
    image.save(modified_image_path)
    print('Secret bits embedded successfully!')

    return modified_image_path

def extract_secret_bits(image_path, key):
    # Load the modified image
    image = Image.open(image_path)
    pixels = image.load()

    # Generate a random seed using the secret key
    random.seed(key)

    secret_bits = []
    width, height = image.size
    for y in range(height):
        for x in range(width):
            r, g, b = pixels[x, y]

            # Extract the LSBs from each color channel
            secret_bits.append(r & 1)
            secret_bits.append(g & 1)
            secret_bits.append(b & 1)

    print('Secret bits extracted successfully!')

    return secret_bits

# Example usage
original_image_path = 'F:/Research/Sahu Sir/output.png'
secret_message = 'Secret message to be embedded'
secret_key = 'Secret key known only to the recipient'

# Convert the secret message to binary representation
secret_bits = ''.join(format(ord(char), '08b') for char in secret_message)

# Embed the secret bits into the image
modified_image_path = embed_secret_bits(original_image_path, secret_bits, secret_key)

# Extract the secret bits from the modified image
extracted_secret_bits = extract_secret_bits(modified_image_path, secret_key)

# Convert the extracted secret bits back to the original message
extracted_message = ''.join(chr(int(extracted_secret_bits[i:i+8], 2)) for i in range(0, len(extracted_secret_bits), 8))

print('Extracted Message:', extracted_message)

ojas

In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad

# Generate a secret key (128-bit key for AES-128)
secret_key = b'16-byte secret key'

# Encrypt the secret message
def encrypt_message(message):
    cipher = AES.new(secret_key, AES.MODE_ECB)
    padded_message = pad(message, AES.block_size)
    ciphertext = cipher.encrypt(padded_message)
    return ciphertext

secret_message = b'Secret message to be encrypted'
encrypted_message = encrypt_message(secret_message)


In [ ]:
# Convert the 5x5 matrix of pixels and encrypted message into binary representations
def to_binary(image):
    binary_image = []
    for row in image:
        binary_row = []
        for pixel in row:
            binary_pixel = format(pixel, '08b')  # Convert pixel to 8-bit binary
            binary_row.append(binary_pixel)
        binary_image.append(binary_row)
    return binary_image

binary_image = to_binary(original_image)
binary_message = ''.join(format(byte, '08b') for byte in encrypted_message)

# Embed the encrypted message into the matrix of pixels using LSB substitution
def embed_message(image, message):
    modified_image = image.copy()
    bit_index = 0
    for row in range(5):
        for col in range(5):
            pixel = modified_image[row][col]
            pixel = list(pixel)
            if bit_index < len(message):
                pixel[-1] = message[bit_index]
                bit_index += 1
            modified_image[row][col] = ''.join(pixel)
    return modified_image

steganographic_image = embed_message(binary_image, binary_message)


In [ ]:
# Extract the hidden message from the steganographic image
def extract_message(image):
    extracted_message = ""
    for row in range(5):
        for col in range(5):
            pixel = image[row][col]
            extracted_message += pixel[-1]
    return extracted_message

extracted_message = extract_message(steganographic_image)

# Decrypt the extracted message using the secret key
def decrypt_message(ciphertext):
    cipher = AES.new(secret_key, AES.MODE_ECB)
    padded_message = cipher.decrypt(ciphertext)
    original_message = padded_message.rstrip(pad(b'', AES.block_size))
    return original_message

decrypted_message = decrypt_message(bytes([int(extracted_message[i:i+8], 2) for i in range(0, len(extracted_message), 8)]))
